# Objectifs

- **manipulation de textes** en vue de leur préparation pour une tâche d'apprentissage automatique : manipuler du texte "authentique" français produit par des internautes, pour en produire des représentations qui mettent en oeuvre une réduction des variables textuelles à différents degrés d'agressivité ;

- **apprentissage automatique** : entraîner et évaluer des modèles de classification automatique des textes préparés ;

- **maîtrise d'outils** : utilisation de librairies qui implémentent des fonctionnalités de manipulation de texte (`spacy`, `nltk`), de données tabulaires (`pandas`) ou d'apprentissage automatique (`scikit-learn`).

# Description de la tâche

__Tâche :__ classification (catégorisation) multi-classe (trois classes)

__Output :__ polarité du texte: positif, négatif, neutre

__Type de données :__ texte, tweets

__Langue :__ français

# (Installation et) importation des outils nécessaires

In [2]:
import os
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
from pprint import pprint

In [3]:
# nltk : en général limité quant au traitement du français
import nltk
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.stem import SnowballStemmer # ou: from nltk.stem.snowball import FrenchStemmer

In [4]:
# En ligne de commande
# !pip install -U spacy
## !python -m spacy download fr
# !python -m spacy download fr_core_news_sm

In [5]:
# spacy : bonne couverture du français ; conçu spécifiquement pour s'interfacer avec des frameworks de deep learning
import spacy
nlp = spacy.load('fr')

**ATTENTION : Les données de test sont destinées à une évaluation FINALE du modèle entraîné. Elles ne doivent sous aucune forme servir à la conception du modèle. Il est donc interdit de les examiner. La sélection itérative du meilleur modèle se fera en évaluant les différentes variantes du modèle (en faisant varier des hyper-paramètres) sur un jeu de données de validation.**

# (A) Récupération et mise en forme des données

Données du défi [DEFT2015](https://deft.limsi.fr/2015/corpus.fr.php?lang=fr) : tweets rédigés en français, portant sur la thématique des changements climatiques. Tweets annotés selon leur polarité, pour la tâche 1 du défi : "Classification des tweets selon leur polarité. Étant donné un tweet, cette tâche consiste à le classer, selon l’opinion/sentiment/émotion qu'il exprime, en positif, négatif, neutre ou mixte (si le tweet contient à la fois un sentiment positif et un sentiment négatif)."

__ATTENTION : Ces jeux de données ont été mis à notre disposition EXCLUSIVEMENT à des fins pédagogiques par les organisateurs du défi DEFT 2015. Leur redistribution est formellement interdite et tout travail écrit (rapport de stage, article, etc.) produit sur la base de ces données devra citer les sources indiquées sur le [site Web de DEFT 2015](https://deft.limsi.fr/2015/corpus.fr.php?lang=fr).__

Les étiquettes (la vérité terrain) et le texte des tweets sont stockés séparément. Nous les regrouperons à partir de l'identifiant du tweet.

## 1. Étiquettes

In [6]:
def make_label_file_path(parent_label_directory, train_or_test):
    return os.path.join(parent_label_directory,
                        '{}_References'.format(train_or_test.title()),
                        'T1.txt')

In [7]:
def map_label_to_numeric(label):
    return 1 if label == '+' else 0 if label == '=' else -1

In [8]:
def get_labels(parent_label_directory, train_or_test):
    label_file = make_label_file_path(parent_label_directory, train_or_test)
    labels = pd.read_table(label_file, header=None, names=['id', 'polarity'])
    labels['polarity'] = labels['polarity'].apply(map_label_to_numeric)
    labels.set_index('id', inplace=True)
    return labels

In [9]:
train_labels = get_labels('data', 'train')
test_labels = get_labels('data', 'test')

In [10]:
train_labels.head()

,polarity
id,
487349133460918272,1
487354248959918080,-1
487360225654374401,0
487387097222098944,-1
487387321537269761,0


In [11]:
train_labels.shape

(7929, 1)

## 2. Textes

In [12]:
def make_data_dir_path(parent_data_directory, train_or_test):
    return os.path.join(parent_data_directory,
                        'deft2015_{}_twitter_raw'.format(train_or_test.upper()))

In [13]:
def get_tweets(parent_data_directory, train_or_test):
    if train_or_test == 'test':
        train_or_test += 's' # incohérence dans le nommage des répertoires
    data_dir = make_data_dir_path(parent_data_dir, train_or_test)
    tweets = dict()

    for file_name in sorted(os.listdir(data_dir)):
        if file_name.endswith(".txt"):
            with open(os.path.join(data_dir, file_name), 'r') as f:
                text = f.read()
            id_tweet = int(os.path.splitext(file_name)[0])
            tweets[id_tweet] = text
    
    return (pd.DataFrame.from_dict(tweets, orient='index')
                        .rename(columns={0: 'text'}))

In [14]:
parent_data_dir = 'data/twitter'
train_tweets = get_tweets(parent_data_dir, 'train')
test_tweets = get_tweets(parent_data_dir, 'test')

In [15]:
train_tweets.head()

,text
487349133460918272,#Question orale à @RoyalSegolene au sujet de l...
487354248959918080,@PhanEd @AznAlainT @_Baekholic alors j'ai une ...
487360225654374401,"""On peut vendre du vent, regarde les éoliennes""\n"
487387097222098944,Développement durable ma gueule\n
487387321537269761,Quand l'cosystme numrique bordelais rencontre ...


__*QUESTION : Combien de tweets y a-t-il dans le jeu de données d'entraînement et dans celui de test ?*__

In [16]:
len(train_tweets), len(test_tweets)

(7511, 3285)

## 3. Jonction des textes et des étiquettes

Il y a plus d'étiquettes que de textes, car des tweets ont pu disparaître entre le moment où ils ont été collectés pour l'annotation de référence et le moment où ils ont été récupérés ultérieurement (voir [ici](https://deft.limsi.fr/2015/evaluation.fr.php?lang=fr)). Nous ferons une jointure interne pour ne retenir que les éléments communs aux deux tableaux.

Cependant, tous les tweets disponibles ont une étiquette :

In [17]:
len(set(train_tweets.index).intersection(set(train_labels.index))) == len(train_tweets)

True

In [18]:
def merge_tweets_and_labels(tweets_df, labels_df):
    return pd.merge(tweets_df, labels_df, how='inner',
                    left_index=True, right_index=True)

In [19]:
train_tweets = merge_tweets_and_labels(train_tweets, train_labels)
test_tweets = merge_tweets_and_labels(test_tweets, test_labels)

In [20]:
train_tweets.head()

,text,polarity
487349133460918272,#Question orale à @RoyalSegolene au sujet de l...,1
487354248959918080,@PhanEd @AznAlainT @_Baekholic alors j'ai une ...,-1
487360225654374401,"""On peut vendre du vent, regarde les éoliennes""\n",0
487387097222098944,Développement durable ma gueule\n,-1
487387321537269761,Quand l'cosystme numrique bordelais rencontre ...,0


## 4. Exploration des données

### 4.1. Distribution des classes

In [21]:
class_distribution = (pd.DataFrame.from_dict(Counter(train_tweets.polarity.values),
                                             orient='index')
                                  .rename(columns={0: 'num_examples'}))
class_distribution.index.name = 'class'

In [22]:
class_distribution

,num_examples
class,
1,2364
-1,1763
0,3384


In [23]:
class_distribution['perc_examples'] = np.around(class_distribution.num_examples /
                                                np.sum(class_distribution.num_examples), 2)

In [24]:
class_distribution

,num_examples,perc_examples
class,,
1,2364,0.31
-1,1763,0.23
0,3384,0.45


### 4.2. Exploration du texte

**ATTENTION :** Ne pas regarder les données de test !

In [25]:
train_tweets['text'].values[:15]

array([ "#Question orale à @RoyalSegolene au sujet de l'efficacité énergétique de @MicheleBonneton http://t.co/RrHxGn5URS @Dailymotion\n",
       "@PhanEd @AznAlainT @_Baekholic alors j'ai une blague mais dure a comprendre: On est ecologiques a nous 3 on se rebelle groupe Anti-Train !\n",
       '"On peut vendre du vent, regarde les éoliennes"\n',
       'Développement durable ma gueule\n',
       "Quand l'cosystme numrique bordelais rencontre la mission French Tech nationale http://t.co/oCqL9tb4SZ\n",
       'Madame Ségolène Royale, ministre de l\'écologie et du développement durable : Non, à la destruction des "nuisibles" http://t.co/jUYWddlIMe\n',
       'Le nouveau Monsieur " développement durable ": Jacques Tapin, l’ex-élu municipal niortais, vient d’être porté ... http://t.co/hrDGKOkyJd\n',
       'Le ciment s’offre une empreinte carbone réduite: Si le ciment est connu pour ses qualités écologiques, il est ... http://t.co/Gd028bTRe2\n',
       "J'ai mis à jour mon profil Viadeo :

# (B) Représentation des textes

## Descripteurs

**Du texte au numérique:**

- les descripteurs (variables, features, traits) sont des unités textuelles (lemmes, racines, autres ; prises individuellement ou en séquences, mais sans égard à leur ordre ou relations : c'est l'approche en "sac de mots") ;

- les valeurs de ces variables sont numériques : binaires, numériques discrètes, numériques continues.

## Texte vs corpus

Traditionnellement : les descripteurs textuels sont calculés sur l'ensemble du corpus. Tous les textes sont représentés par le mêmes ensemble de descripteurs, ce qui fait que la représentation d'un texte est un grand vecteur épars de taille fixe (taille du vecteur = taille du vocabulaire du corpus).

Approches appliquées dans l'état de l'art : chaque mot est représenté par un vecteur dense de valeurs réelles. Le texte est représenté par une aggrégation sous une certaine forme des représentations de ses mots constituants.

## 1. Sélection de descripteurs : prétraitements textuels

Objectif : réduire le nombre de descripteurs : réduire à un seul descripteur ceux qui sont équivalents (p. ex. deux mots qui ont été écrits avec et sans accents respectivement) ou qui peuvent être regroupés dans une classe d'équivalence (p. ex. remplacer toutes les instances de date par un mot fictif, p. ex. "DATEEXPR").

Quelques procédés : lemmatisation, racinisation, normalisation/correction orthographique, suppression des accents, mise en minuscules, suppression de la ponctuation, suppression de certains mots (mots dits "vides", autres mots), substitution de certains mots par un autre représentant leur appartenance à une classe, etc.

Dans la pratique certains ces procédés sont souvent appliqués ensemble ou bien ils peuvent être pris en charge par la boîte à outils d'apprentissage automatique (via la spécification de divers paramètres à certaines étape du processus), qui les applique alors en boîte noire. D'autre part, le choix d'appliquer ou non un certain procédé doit prendre en compte les besoins du contexte concret (p. ex. une mise en minuscules affecte-t-elle la reconnaissance d'entités nommées, si celle-ci est préconisée ?). Il n'y a pas de recette ! Mais on peut s'intéresser à ce qui a marché pour d'autres (en consultant la littérature).

Comme entraînement à la manipulation du texte, nous verrons quelques exemples de transformation du texte. Nous produirons plusieurs versions de nos textes, qui pourront servir par la suite à l'étape d'apprentissage du classifieur et de prédiction.

Pour illustrer l'effet des différentes transformations sur le texte, prenons comme exemple ce tweet:

In [217]:
# Exemple
tw = train_tweets['text'].iloc[100]
tw_nlp = nlp(tw)
tw

'#Bruxelles boude le #Fonds vert pour le #climat par @EuroActiv_FR | @Actuenviro http://t.co/eFKkE9W0GI\n'

### 1.1. Pas de sélection : mots tels quels

In [40]:
for token in tw_nlp:
    print(token)

#
Bruxelles
boude
le
#
Fonds
vert
pour
le
#
climat
par
@EuroActiv_FR
|
@Actuenviro
http://t.co/eFKkE9W0GI




### 1.2. Réduction par regroupement/uniformisation

#### 1.2.1. Lemmes

In [41]:
for token in tw_nlp:
    print(token.lemma_)

#
bruxelles
bouder
le
#
fonds
vert
pour
le
#
climat
par
@euroactiv_fr
|
@actuenviro
http://t.co/efkke9w0gi




**EXERCICE.** Produire une version lemmatisée des tweets et la mettre dans une colonne `lemmas` dans la dataframe. Pour ce faire : créer une fonction qui lemmatise un texte ; appliquer cette fonction à la colonne `text` des deux dataframes (`train` et `test`). Attention, le traitement de toute la colonne peut prendre un peu de temps.

In [243]:
def lemmatise_text(text):
    text = nlp(text)
    lemmas = [token.lemma_ for token in text]
    return ' '.join(lemmas)

In [244]:
lemmatise_text(tw)

'# bruxelles bouder le # fonds vert pour le # climat par @euroactiv_fr | @actuenviro http://t.co/efkke9w0gi \n'

In [230]:
train_tweets['lemmas'] = train_tweets['text'].apply(lemmatise_text)

In [232]:
train_tweets.head()

,text,polarity,lemmas
487349133460918272,#Question orale à @RoyalSegolene au sujet de l...,1,# question oral à @royalsegolene au sujet de l...
487354248959918080,@PhanEd @AznAlainT @_Baekholic alors j'ai une ...,-1,@phaned @aznalaint @_baekholic alors il avoir ...
487360225654374401,"""On peut vendre du vent, regarde les éoliennes""\n",0,""" on pouvoir vendre du vent , regarder le éoli..."
487387097222098944,Développement durable ma gueule\n,-1,développement durable mon gueuler \n
487387321537269761,Quand l'cosystme numrique bordelais rencontre ...,0,quand le cosystme numrique bordelais rencontre...


In [226]:
test_tweets['lemmas'] = test_tweets['text'].apply(lemmatise_text)

In [233]:
test_tweets.shape

(3283, 3)

#### 1.2.2. Racines

In [42]:
stemmer = SnowballStemmer('french')
tokenizer = TweetTokenizer()
tokenizer.tokenize(tw)

['#Bruxelles',
 'boude',
 'le',
 '#Fonds',
 'vert',
 'pour',
 'le',
 '#climat',
 'par',
 '@EuroActiv_FR',
 '|',
 '@Actuenviro',
 'http://t.co/eFKkE9W0GI']

Les tweets présentent des particularités par rapport à d'autres textes. Des outils conçus spécifiquement pour gérer ce type de texte existent. Par exemple, `nltk` propose un tokeniseur pour les tweets:

In [52]:
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True) 
# strip_handles supprime les @...
# reduce_len réduit les séquences de caractères répétés plus de trois fois à des séquences de taille trois
tokenizer.tokenize(tw)

['#Bruxelles',
 'boude',
 'le',
 '#Fonds',
 'vert',
 'pour',
 'le',
 '#climat',
 'par',
 '|',
 'http://t.co/eFKkE9W0GI']

In [53]:
for token in tokenizer.tokenize(tw):
    print(stemmer.stem(token))

#bruxel
boud
le
#fond
vert
pour
le
#climat
par
|
http://t.co/efkke9w0g


**EXERCICE.** Produire une version racinisée des tweets et la mettre dans une colonne `stems` dans la dataframe. Pour ce faire : créer une fonction qui racinise un texte ; appliquer cette fonction à la colonne `text` des deux dataframes (`train` et `test`).

In [245]:
def stem_text(text):
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    stemmer = SnowballStemmer('french')
    stems = [stemmer.stem(token) for token in tokenizer.tokenize(text)]
    return ' '.join(stems)

In [246]:
stem_text(tw)

'#bruxel boud le #fond vert pour le #climat par | http://t.co/efkke9w0g'

In [237]:
train_tweets['stems'] = train_tweets['text'].apply(stem_text)

In [238]:
train_tweets.head()

,text,polarity,lemmas,stems
487349133460918272,#Question orale à @RoyalSegolene au sujet de l...,1,# question oral à @royalsegolene au sujet de l...,#question oral à au sujet de l'efficac énerget...
487354248959918080,@PhanEd @AznAlainT @_Baekholic alors j'ai une ...,-1,@phaned @aznalaint @_baekholic alors il avoir ...,alor j'ai une blagu mais dur a comprendr : on ...
487360225654374401,"""On peut vendre du vent, regarde les éoliennes""\n",0,""" on pouvoir vendre du vent , regarder le éoli...",""" on peut vendr du vent , regard le éolien """
487387097222098944,Développement durable ma gueule\n,-1,développement durable mon gueuler \n,développ durabl ma gueul
487387321537269761,Quand l'cosystme numrique bordelais rencontre ...,0,quand le cosystme numrique bordelais rencontre...,quand l'cosystm numriqu bordel rencontr la mis...


In [239]:
test_tweets['stems'] = test_tweets['text'].apply(stem_text)

In [240]:
test_tweets.shape

(3283, 4)

#### 1.2.3. Étiquettes morphosyntaxiques

In [59]:
for token in tw_nlp:
    print(token.pos_)

ADJ
PROPN
VERB
DET
NOUN
NOUN
ADJ
ADP
DET
NOUN
NOUN
ADP
NOUN
ADJ
AUX
AUX
SPACE


**EXERCICE.** Produire une version des tweets où chaque token est remplacé par son étiquette morphosyntaxique et la mettre dans une colonne `pos` dans la dataframe. Pour ce faire : créer une fonction qui remplace les mots par leurs étiquettes dans un texte ; appliquer cette fonction à la colonne `text` des deux dataframes (`train` et `test`). Attention, le traitement de toute la colonne peut prendre un peu de temps.

In [247]:
def replace_words_with_pos_tag(text):
    text = nlp(text)
    return ' '.join([token.pos_ for token in text])

In [248]:
replace_words_with_pos_tag(tw)

'ADJ PROPN VERB DET NOUN NOUN ADJ ADP DET NOUN NOUN ADP NOUN ADJ AUX AUX SPACE'

In [249]:
train_tweets['pos'] = train_tweets['text'].apply(replace_words_with_pos_tag)

In [251]:
train_tweets.head()

,text,polarity,lemmas,stems,pos
487349133460918272,#Question orale à @RoyalSegolene au sujet de l...,1,# question oral à @royalsegolene au sujet de l...,#question oral à au sujet de l'efficac énerget...,PRON NOUN ADJ ADP PROPN PRON NOUN ADP DET NOUN...
487354248959918080,@PhanEd @AznAlainT @_Baekholic alors j'ai une ...,-1,@phaned @aznalaint @_baekholic alors il avoir ...,alor j'ai une blagu mais dur a comprendr : on ...,ADP DET NOUN ADV PRON VERB DET NOUN CCONJ ADJ ...
487360225654374401,"""On peut vendre du vent, regarde les éoliennes""\n",0,""" on pouvoir vendre du vent , regarder le éoli...",""" on peut vendr du vent , regard le éolien """,PUNCT PRON AUX VERB DET NOUN PUNCT CCONJ DET N...
487387097222098944,Développement durable ma gueule\n,-1,développement durable mon gueuler \n,développ durabl ma gueul,NOUN ADJ DET NOUN SPACE
487387321537269761,Quand l'cosystme numrique bordelais rencontre ...,0,quand le cosystme numrique bordelais rencontre...,quand l'cosystm numriqu bordel rencontr la mis...,SCONJ DET NOUN ADJ ADJ VERB DET NOUN PROPN PRO...


In [252]:
test_tweets['pos'] = test_tweets['text'].apply(replace_words_with_pos_tag)

In [253]:
test_tweets.shape

(3283, 5)

#### 1.2.4. Classe d'appartenance des entités nommées

Limites : reconnaissance imparfaite. Faire des essais pour appréhender les limites de l'outil. Cela nous permettra, par exemple, de corriger certaines erreurs systématiques de l'outil en intervenant en amont sur le texte pour transformer les éléments qui posent difficulté. Par exemple, dans notre cas, l'outil semble ne pas bien gérer les URL. On peut donc penser à les normaliser avant d'appliquer la reconnaissance d'entités nommées.

In [254]:
tw_nlp.ents

(Bruxelles, Fonds vert, | @Actuenviro, )

In [255]:
[ent.label_ for ent in tw_nlp.ents]

['LOC', 'ORG', 'MISC', 'MISC']

Analyse des erreurs : la chaîne vide a été identifiée comme entité de la classe MISC ; la troisième entité inclut un caractère séparateur suivi d'un espace. Ces erreurs pourraient probablement être évitées par un nettoyage du texte en amont.

**EXERCICE.** Produire une version des tweets où chaque entité nommée identifiée est remplacée par sa catégorie et la mettre dans une colonne `entities` dans la dataframe. Pour ce faire : créer une fonction qui remplace les entités par leurs étiquettes dans un texte ; appliquer cette fonction à la colonne `text` des deux dataframes (`train` et `test`). Attention, le traitement de toute la colonne peut prendre un peu de temps.

In [258]:
[token.label_ if token in tw_nlp.ents else token for token in tw_nlp]

TypeError: Argument 'other' has incorrect type (expected spacy.tokens.token.Token, got spacy.tokens.span.Span)

In [262]:
for token in tw_nlp:
    if token.text in tw_nlp.ents:
        print(token.label_)
    else:
        print(token)

TypeError: Argument 'other' has incorrect type (expected spacy.tokens.span.Span, got str)

In [ ]:
def replace_entities(text):
    text = nlp(text)
    entities = text.ents
    [token.label_ else token if token in entities for token in text]

#### 1.2.5. Autres classes

Comme exemple, nous remplacerons les adresses Web par un mot fictif URLEXPR. N'hésitez pas à penser à d'autres classes d'équivalence qui vous semblent pertinentes !

**EXERCICE.** Créer une fonction `substitute_url` qui prend en entrée une chaîne de caractères et le mot de remplacement (p. ex. "URLEXPR") et remplace les URL présentes dans la chaîne de caractères par le mot de remplacement donné en argument. Indication : utiliser des expressions régulières (module `re`) ; examiner des exemples de tweets (du corpus 
d'entraînement) pour bien saisir la structure des URL. Appliquer ensuite cette fonction à la colonne `text` des dataframes d'entraînement et de test. Dans les deux cas, stocker le résultat de la transformation dans une nouvelle colonne `url`.

### 1.3. Réduction par filtrage : suppression de certains mots

#### 1.3.1. Filtrage des mots par fréquence d'utilisation en langue générale : "mots vides"

Critère vague. La notion de mot vide peut varier selon le contexte. En général géré lors de la création de la matrice documents-termes (voir plus bas, pour le calcul des valeurs des descripteurs).

#### 1.3.2. Filtrage des mots par contenu expressif : mots qui n'ont pas une polarité claire

Nous reviendrons sur ce procédé quand nous aborderons les baselines pour notre tâche de classification.

#### Moyennant un dictionnaire

Par exemple : dictionnaire [FEEL](http://advanse.lirmm.fr/feel.php) (voir [article](https://hal-lirmm.ccsd.cnrs.fr/lirmm-01348016/document)) : liste de 14128 **lemmes** annotés en termes de polarité (positif/négatif) et de six émotions (joy, fear, sadness, anger, surprise, disgust).

#### En exploitant les fonctionnalités d'un outil de TAL

Certains outils, dont `spacy`, peuvent attacher un score de sentiment aux tokens.

## 2. Calcul des valeurs des descripteurs

Avant de procéder aux calculs, nous séparerons un jeu de données de validation à partir des données d'entraînement initiales. Nous mettons les données de test fournies de côté, exclusivement pour une évaluation finale des modèles.

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_tweets['text'],
                                                      train_tweets['polarity'],
                                                      train_size=0.75,
                                                      random_state=5)

/home/gizmo/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [80]:
X_train.shape, X_valid.shape

((5633,), (1878,))

In [97]:
X_test, y_test = test_tweets['text'], test_tweets['polarity']

### 2.1. Binaire : présence/absence

### 2.2. Numérique discret : décomptes

In [66]:
from sklearn.feature_extraction.text import CountVectorizer

In [67]:
CountVectorizer?

Étudier la documentation du constructeur. De nombreuses options de réduction du vocabulaire sont proposées. En voici quelques-unes :

- suppression des accents : `strip_accents` ;

- mise en minuscule : `lowercase` (par défault `True`) ;

- seuillage sur la fréquence documentaire (c.à.d. le nombre de documents dans lesquels le terme apparaît) ; exemple : `max_df=0.7` signifie qu'on ignore les termes qui sont présentes dans plus de 70% des textes du corpus (ce qui équivaut à l'élimination des mots vides propres au corpus) ; `min_df=5` ignore les termes qui apparaissent dans moins de 5 textes du corpus ;

- seuillage du nombre de variables à retenir : `max_features=1000` ne retient que les 1000 termes qui ont les "term frequency" (nombre d'occurrences dans un texte particulier) les plus élevées ;

- suppression de mots vides : `stop_words` (liste par défaut ou fournie) ;

- ordre des n-grammes : `ngram_range=(min_n, max_n)` extrait les n-grammes dont la taille est entre `min_n` et `max_n`.

In [152]:
# Calcul des fréquences d'occurrence des termes dans le corpus, avec les options par défaut
vect_count = CountVectorizer().fit(X_train)

In [153]:
vect_count

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

Examinons le vocabulaire de notre corpus:

In [154]:
vect_count.get_feature_names()[:100]

['00',
 '000',
 '01',
 '013',
 '01business_fr',
 '01iwiyyjyj',
 '01kcbtpxur',
 '01net',
 '02',
 '03',
 '034jkracav',
 '03gvexadww',
 '03vuozibff',
 '04',
 '05',
 '0539oglj1k',
 '06',
 '07',
 '077707',
 '07la',
 '08',
 '08ovazgsvp',
 '08xaecp2qb',
 '09',
 '09pgnr35va',
 '09ushmifhr',
 '0agsmeki3s',
 '0ar2oto063',
 '0bdfmc9xye',
 '0bevqostpr',
 '0c0dgfcl3n',
 '0cbub6cr8l',
 '0cqryb3www',
 '0dbeqd7q64',
 '0dv74sttev',
 '0ed5jl9dsr',
 '0eip8htosr',
 '0em2kxoym1',
 '0epbnsrqfs',
 '0fftvqzrkz',
 '0gcaqeuxga',
 '0gzv2i2f4p',
 '0hu6xafz3u',
 '0husujgd5b',
 '0hyt8ayhai',
 '0iqdmldjxz',
 '0jyqssje6d',
 '0k7vgj8vsh',
 '0l0iwtbj81',
 '0lfq23fxux',
 '0lwhyffmak',
 '0mamllnhqy',
 '0msqmx44xw',
 '0napsdwx4w',
 '0nmate58kc',
 '0nwxmzvjex',
 '0ofvoxacdy',
 '0okkgzqrho',
 '0om4showzg',
 '0oqsf3kbcn',
 '0qtf3xttzc',
 '0ssbgvyopk',
 '0tjsewjv1a',
 '0u68bgwyry',
 '0uh1umaeiq',
 '0uosjy9thu',
 '0us6f1caqt',
 '0uyvnhzk90',
 '0vjeeewvmo',
 '0w1aptxpzw',
 '0wtuv9jixw',
 '0xafumcdor',
 '0xeawyhewf',
 '0xqldwu9d

In [155]:
vect_count.get_feature_names()[-100:]

['équilibre',
 'équipe',
 'équipements',
 'équipent',
 'équipée',
 'équipés',
 'équitable',
 'équitables',
 'équiterre',
 'équivalent',
 'équivoque',
 'ér',
 'érable',
 'érection',
 'érige',
 'éro',
 'érosion',
 'éruptions',
 'és',
 'ésente',
 'établi',
 'établir',
 'étage',
 'étaient',
 'étais',
 'était',
 'étals',
 'étant',
 'étape',
 'étapes',
 'état',
 'états',
 'éteindre',
 'éteintes',
 'étendent',
 'étendre',
 'éthanol',
 'éthiopie',
 'éthique',
 'éthiques',
 'étienne',
 'étique',
 'étoffé',
 'étoile',
 'étonnant',
 'étonnante',
 'étonne',
 'étonnes',
 'étonnée',
 'étourdissant',
 'étrange',
 'étrangère',
 'étrangères',
 'étre',
 'étroit',
 'étroites',
 'étude',
 'études',
 'étudiant',
 'étudiante',
 'étudiants',
 'étudie',
 'été',
 'éunion',
 'évacuation',
 'évaluation',
 'évaluer',
 'évangé',
 'évangéliste',
 'éveil',
 'éveloppement',
 'évidemment',
 'évidence',
 'évident',
 'évier',
 'éviter',
 'évolue',
 'évoluent',
 'évoluer',
 'évolution',
 'évolutions',
 'évoquant',
 'évoq

In [156]:
len(vect_count.get_feature_names()) # taille du vocabulaire

17673

In [157]:
# Création de la matrice document-termes
X_train_vectorized_count = vect_count.transform(X_train)
X_train_vectorized_count

<5633x17673 sparse matrix of type '<class 'numpy.int64'>'
	with 84342 stored elements in Compressed Sparse Row format>

In [158]:
# Le corpus de validation et celui de test doivent être également transformées en matrice document-termes. Les termes sont ceux décomptés sur le corpus d'entraînement. Les termes présent dans le corpus de validation ou de test mais absents du corpus d'entraînement seront ignorés.
X_valid_vectorized_count = vect_count.transform(X_valid)
X_test_vectorized_count = vect_count.transform(X_test)

Cette fois-ci nous allons inclure des bigrammes dans le vocabulaire:

In [194]:
vect_count_bigrams = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)
X_train_vectorized_count_bigrams = vect_count_bigrams.transform(X_train)
X_valid_vectorized_count_bigrams = vect_count_bigrams.transform(X_valid)
X_test_vectorized_count_bigrams = vect_count_bigrams.transform(X_test)
len(vect_count_bigrams.get_feature_names())

3920

### 2.3. Numérique continu : TF-IDF (ou autres pondérations diverses)

In [169]:
from sklearn.feature_extraction.text import TfidfVectorizer

Limitons le vocabulaire à des termes qui apparaissent dans au moins 5 documents.

In [170]:
vect_tfidf = TfidfVectorizer(min_df=5).fit(X_train)

La réduction de la taille du vocabulaire est spectaculaire !

In [172]:
len(vect_count.get_feature_names()), len(vect_tfidf.get_feature_names())

(17673, 2039)

In [174]:
X_train_vectorized_tfidf = vect_tfidf.transform(X_train)
X_valid_vectorized_tfidf = vect_tfidf.transform(X_valid)
X_test_vectorized_tfidf = vect_tfidf.transform(X_test)

# Classification des textes

Nous entraînerons des modèles de classification appartenant à quelques familles d'algorithmes d'apprentissage automatique. D'autres familles restent à explorer. L'objectif est de comparer non seulement les performances des différentes méthodes entre elles, mais aussi la performance d'une même méthode sur des représentations différentes du texte.

## Workflow

**Schéma général :** apprentissage (> évaluation sur données de validation > apprentissage > évaluation sur données de validation >...) (> évaluation sur données de test) > prédiction

Boîte à outils (algorithmes pré-implémentés) : `scikit-learn`. Interface unifiée pour l'ensemble des algorithmes. **Mise en oeuvre :**

* si besoin de réduire et/ou centrer les données (réduction statistique) :
    - création d'un objet `scaler` de la classe adaptée (réducteur des données) ;
    - entraînement du `scaler` sur les données d'entraînement : méthode `fit` de l'objet réducteur ;
    - réduction des données d'entraînement : méthode `transform` du réducteur ; cette étape peut être enchaînée avec la précédente grâce à la méthode `fit_transform` du réducteur ;
    - réduction des données de validation et de test : méthode `transform` du réducteur (attention : même réducteur que pour les données d'entraînement ! On ne réapprend pas les critères de réduction sur les données de validation/test !) ;


* création de l'objet estimateur : appel du constructeur de la classe pertinente, avec d'éventuels paramètres si valeurs autres que défaut ;

* apprentissage de l'estimateur sur les données d'entraînement (éventuellement réduites) : méthode `fit` de l'estimateur ; cette étape peut être enchaînée avec la précédente ;

* évaluation de l'estimateur sur les données d'entraînement, de validation et/ou (uniquement si c'est le modèle final !) de test : méthode `score` de l'estimateur.

* prédiction sur des données nouvelles : méthode `predict` de l'estimateur.

**Métriques d'évaluation :**

* taux de bonne classification (*accuracy*) ;

* précision (*precision*) ;

* recall (*rappel*) ;

* score F1 ;

* aire sous la courbe ROC (ROC AUC) : pour la classification binaire ;

* métriques "maison", sur mesure.

**EXERCICE.** Créer une fonction appelée `score_on_dataset`, qui prend en entrée un modèle entraîné, un jeu de données et les étiquettes correspondant à celui-ci, et retourne un dictionnaire contenant les scores du modèle sur le jeu de données. Le dictionnaire aura cette forme :

`{'confusion_matrix': [[..., ...], ...], 'accuracy': 0.85, 'precision': 0.7, 'recall': ..., 'F1_macro': ..., 'F1_micro': ...}`

**EXERCICE.** Créer une fonction qui, en appelant la fonction `score_on_dataset`, évalue le modèle sur les données d'entraînement et de validation, et retourne un dictionnaire de dictionnaires contenant les scores du modèle sur les deux jeux de données. Le dictionnaire aura cette forme :

`{'train': {'confusion_matrix': [[..., ...], ...],
            'accuracy': 0.85,
            'precision': 0.7,
            'recall': ...,
            'F1_macro': ...,
            'F1_micro': ...},
 'validation': {'confusion_matrix': [[..., ...], ...],
                'accuracy': ...,
                'precision': 0.7,
                'recall': ...,
                'F1_macro': ...,
                'F1_micro': ...},
 }`

### 1. Modèles de référence faibles (*weak baselines*)

#### 1.1. Choix aléatoire

Le taux de bonne classification (*accuracy*) est la probabilité de choisir une classe. Toutes les classes ont les mêmes chances d'être choisies.

**EXERCICE.** Quelle est le taux de bonne classification pour cette approche dans notre scénario ?

#### 1.2. Prédiction constante de la classe majoritaire

Seules les instances de la classe majoritaire seront classées correctement.

**EXERCICE.** Quelle est le taux de bonne classification pour cette approche dans notre scénario ?

Vérifiez votre réponse :

In [94]:
from sklearn.dummy import DummyClassifier
maj = DummyClassifier(strategy='most_frequent').fit(X_train_vectorized, y_train)
predictions_valid = maj.predict(vect.transform(X_valid))
predictions_valid

array([0, 0, 0, ..., 0, 0, 0])

In [106]:
maj_class = (class_distribution.index[class_distribution.perc_examples ==
                                      np.amax(class_distribution.perc_examples)][0])
maj_class

0

In [107]:
np.all(predictions_valid == maj_class)

True

In [108]:
maj.score(X_valid_vectorized, y_valid)

0.45527156549520764

### 2. Modèle de référence fort (*strong baseline*) : approche sans apprentissage automatique : méthode du lexique de polarité

Calcul d'un score de polarité pour chaque texte, selon la formule (simplifiée) : nombre_de_termes_positifs - nombre_de_termes_négatifs ([Hu et Liu (2004)](https://pdfs.semanticscholar.org/13e5/f0c40c85ca8e01b3756963d5352358de7c29.pdf), [Kim et Hovy (2004)](http://anthology.aclweb.org/P/P06/P06-2.pdf#page=493)). Cela revient à un filtrage des mots, suivi d'un calcul de score. Des variantes affinées de cette approche ont été proposées.

### 3. Classifieur Naïve Bayes

En général pris également comme baseline.

In [207]:
from sklearn.naive_bayes import MultinomialNB
model_nb = MultinomialNB().fit(X_train_vectorized_tfidf, y_train)
predictions_valid = model_nb.predict(X_valid_vectorized_tfidf)

In [208]:
accuracy_score(y_valid, predictions_valid)

0.626730564430245

In [209]:
print(classification_report(y_valid, predictions_valid))

             precision    recall  f1-score   support

         -1       0.72      0.47      0.57       466
          0       0.60      0.82      0.69       855
          1       0.63      0.46      0.53       557

avg / total       0.64      0.63      0.61      1878



### 4. Régression linéaire

In [175]:
from sklearn.linear_model import LogisticRegression

**EXERCICE.** Entraîner un modèle avec les arguments suivants : `multi_class='multinomial'`, `solver='lbfgs'` sur le corpus vectorisé par nombre d'occurrences et l'évaluer sur le corpus de validation.

In [199]:
model_lr = LogisticRegression(multi_class='multinomial',
                              solver='lbfgs').fit(X_train_vectorized_count, y_train)

In [200]:
predictions_valid = model_lr.predict(X_valid_vectorized_count)

In [163]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [162]:
accuracy_score(y_valid, predictions_valid)

0.66027689030883918

In [164]:
print(classification_report(y_valid, predictions_valid))

             precision    recall  f1-score   support

         -1       0.74      0.54      0.62       466
          0       0.65      0.78      0.71       855
          1       0.62      0.58      0.60       557

avg / total       0.67      0.66      0.66      1878



Examinons les variables (termes) ayant l'association la plus forte avec chaque classe.

In [198]:
def print_n_strongly_associated_features(vectoriser, model, n):
    feature_names = np.array(vectoriser.get_feature_names())

    for i in range(3):
        class_name = model.classes_[i]
        print("CLASSE {}".format(class_name))
        idx_coefs_sorted = model.coef_[i].argsort()
        print("Les dix variables ayant l'association négative la plus forte " + 
              "avec la classe {} :\n{}\n".format(class_name,
                                                 feature_names[idx_coefs_sorted[:n]]))
        print("Les dix variables ayant l'association positive la plus forte " +
              "avec la classe {} :\n{}\n"
              .format(class_name,
                      feature_names[idx_coefs_sorted[:-(n + 1):-1]]))
        print()

In [201]:
print_n_strongly_associated_features(vect_count, model_lr, 10)

CLASSE -1
Les dix variables ayant l'association négative la plus forte avec la classe -1 :
['traitement' 'article' 'photovoltaïques' 'renouvelables' 'grâce' 'peuvent'
 'raison' 'décision' 'réaction' 'comprendre']

Les dix variables ayant l'association positive la plus forte avec la classe -1 :
['menace' 'danger' 'scandale' 'mal' 'acidification' 'menacée' 'gueule'
 'rejette' 'noire' 'crise']


CLASSE 0
Les dix variables ayant l'association négative la plus forte avec la classe 0 :
['contre' 'grande' 'merci' 'veut' 'grand' 'bonne' 'menace' 'bravo' 'espèce'
 'aime']

Les dix variables ayant l'association positive la plus forte avec la classe 0 :
['fois' 'french' 'cap' 'art' 'algérie' '11' 'ni' 'consultation' 'dis'
 'montre']


CLASSE 1
Les dix variables ayant l'association négative la plus forte avec la classe 1 :
['mal' 'piège' 'eoliennes' 'promotion' 'oiseaux' 'communistes' 'fois'
 'danger' 'presse' 'abandon']

Les dix variables ayant l'association positive la plus forte avec la classe 

**EXERCICE.** Entraîner un modèle avec les arguments suivants : `multi_class='multinomial'`, `solver='lbfgs'` sur le corpus vectorisé par TF-IDF et l'évaluer sur le corpus de validation.

In [178]:
model_lr = LogisticRegression(multi_class='multinomial',
                              solver='lbfgs').fit(X_train_vectorized_tfidf, y_train)
predictions_valid = model_lr.predict(X_valid_vectorized_tfidf)

In [179]:
accuracy_score(y_valid, predictions_valid)

0.63099041533546329

In [180]:
print(classification_report(y_valid, predictions_valid))

             precision    recall  f1-score   support

         -1       0.71      0.50      0.58       466
          0       0.63      0.77      0.69       855
          1       0.59      0.54      0.56       557

avg / total       0.64      0.63      0.63      1878



La performance est légèrement inférieure, mais nous l'avons obtenue en utilisant considérablement moins de variables.

In [191]:
feature_names = np.array(vect_tfidf.get_feature_names())
idx_tfidf_sorted = X_train_vectorized_tfidf.max(0).toarray()[0].argsort()
print("TF-IDF le moins élevé : {}".format(feature_names[idx_tfidf_sorted[:10]]))
print("TF-IDF le plus élevé : {}".format(feature_names[idx_tfidf_sorted[:-11:-1]]))

TF-IDF le moins élevé : ['recuperats' '2fkd2lpde9' 'x1js7tox3c' 'culminant' 'bilans' 'aliments'
 '06' 'taller' 'dinar' 'aprofitament']
TF-IDF le plus élevé : ['écologique' 'éoliennes' 'écologie' 'eolienne' 'air' 'ah' 'maroc'
 'écologiques' 'scoopit' 'bonjour']


Avec le vectorisateur à **unigrammes et bigrammes** :

In [202]:
model_lr = LogisticRegression(multi_class='multinomial',
                              solver='lbfgs').fit(X_train_vectorized_count_bigrams,
                                                  y_train)
predictions_valid = model_lr.predict(X_valid_vectorized_count_bigrams)

In [203]:
accuracy_score(y_valid, predictions_valid)

0.626730564430245

In [204]:
print(classification_report(y_valid, predictions_valid))

             precision    recall  f1-score   support

         -1       0.69      0.52      0.60       466
          0       0.64      0.73      0.68       855
          1       0.57      0.56      0.56       557

avg / total       0.63      0.63      0.62      1878



In [205]:
print_n_strongly_associated_features(vect_count_bigrams, model_lr, 10)

CLASSE -1
Les dix variables ayant l'association négative la plus forte avec la classe -1 :
['raison' 'photovoltaïques' 'article' 'énergies renouvelables' 'voir'
 'la chasse' 'pourraient' 'je ne' 'fil' 'vivre']

Les dix variables ayant l'association positive la plus forte avec la classe -1 :
['menace' 'gueule' 'quitte' 'danger' 'noire' 'mal' 'pénurie' 'pire' 'non'
 'crise']


CLASSE 0
Les dix variables ayant l'association négative la plus forte avec la classe 0 :
['menace' 'belles' 'hausse' 'belle' 'veut' 'aime' 'bravo' 'crise'
 'contre le' 'bonne']

Les dix variables ayant l'association positive la plus forte avec la classe 0 :
['consultation' 'ses éoliennes' 'énergétique de' 'arrive' 'de innovation'
 'cours' 'qui se' 'art' '11' 'inter']


CLASSE 1
Les dix variables ayant l'association négative la plus forte avec la classe 1 :
['énergétique de' 'est le' 'frein' 'abandon' 'disparition' 'eoliennes'
 'mal' 'octobre' 'de innovation' 'lepoint']

Les dix variables ayant l'association positiv

### 5. SVM

In [210]:
from sklearn.svm import SVC

In [213]:
model_svm = SVC(kernel='linear', C=0.1).fit(X_train_vectorized_count_bigrams, y_train)
predictions_valid = model_svm.predict(X_valid_vectorized_count_bigrams)

In [214]:
accuracy_score(y_valid, predictions_valid)

0.62087326943556975

In [215]:
print(classification_report(y_valid, predictions_valid))

             precision    recall  f1-score   support

         -1       0.72      0.47      0.57       466
          0       0.60      0.82      0.69       855
          1       0.61      0.45      0.52       557

avg / total       0.63      0.62      0.61      1878



## Pistes pour l'approfondissement (travail personnel)

**1.** Pousser la réduction de variables plus loin : correction d'orthographe, autres transformations considérées pertinentes. Il n'y a pas de recette qui vaille partout, il faut essayer différentes approches et voir ce qui marche le mieux sur nos textes et notre tâche. Le texte des tweets pose des problèmes particulier, il faut donc y adapter les traitements.

**2.** Combiner des descripteurs textuels avec des variables non-textuelles. Il faudra construire explicitement la matrice document-termes (et appliquer probablement un filtrage plus agressif, pour des raisons de coût de mémoire) pour pouvoir l'augmenter d'autres variables. Le fichier `json` correspondant à chaque tweet contient des méta-données. En choisir une ou deux, les extraire et les ajouter à la représentation des données. Puis entraîner un modèle de classification sur ce nouveau jeu de descripteurs.

**3.** Essayer d'autres algorithmes de classification, par exemple : arbres de décision (`from sklearn.tree import DecisionTreeClassifier`), méthodes d'ensemble (forêts aléatoires : `from sklearn.ensemble import RandomForestClassifier` ; gradient-boosted decision trees : `from sklearn.ensemble import GradientBoostingClassifier`), réseaux de neurones simple (`from sklearn.neural_network import MLPClassifier`) ou bien des architectures plus complexes (autres librairies : `tensorflow`, `keras`, etc.).

**4.** Ajuster les hyper-paramètres d'un modèle par validation croisée (`from sklearn.model_selection import GridSearchCV`). Utiliser dans ce cas la totalité du jeu d'entraînement fourni initialement (ne plus en séparer une portion pour la validation).